<a href="https://colab.research.google.com/github/IngeborgOlssoen/Chagas-challenge-Hear-heart/blob/main/Chagas_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount to google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Locate repo

In [4]:
%cd /content/drive/MyDrive/chagas-challenge
!ls



/content/drive/MyDrive/chagas-challenge
base_model.py	   prepare_code15_data.py    run_model.py
Dockerfile	   prepare_ptbxl_data.py     scores.csv
evaluate_model.py  prepare_samitrop_data.py  script.py
helper_code.py	   __pycache__		     splitting_data.py
LICENSE		   README.md		     team_code.py
model		   requirements.txt	     train_model.py


# Installing and imports

## Installing

In [5]:
!pip install wfdb
!pip install -r requirements.txt
!pip install wfdb scikit-learn numpy pandas torch torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93

## Imports

In [6]:
import wfdb
import os
import zipfile
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import os
import shutil
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import WeightedRandomSampler, DataLoader
from team_code import ECGFromCSV
from helper_code import load_header, get_label, get_source, get_age, get_sex

DATA_DIR = "/content/drive/MyDrive/chagas_datasets"


# Dataset fix

In [7]:


# Create the directory if it doesn't exist

os.makedirs(DATA_DIR, exist_ok=True)

#Download  if not already downloaded

ptbxl_path = os.path.join(DATA_DIR, "ptbxl_output.zip")
samitrop_path = os.path.join(DATA_DIR, "samitrop_output.zip")


if not os.path.exists(ptbxl_path):
    print("PTB-XL dataset not found...")
else:
    print("PTB-XL dataset already downloaded.")

if not os.path.exists(samitrop_path):
    print("Samitrop dataset not found...")
else:
    print("Samitrop dataset already downloaded.")


PTB-XL dataset not found...
Samitrop dataset not found...


In [8]:


# Paths to the zip files and extract directories
ptbxl_path = os.path.join(DATA_DIR, "ptbxl_output.zip")
samitrop_path = os.path.join(DATA_DIR, "samitrop_output.zip")
ptbxl_extract_path = os.path.join(DATA_DIR, "ptbxl_output")
samitrop_extract_path = os.path.join(DATA_DIR, "samitrop_output")

# Check if PTB-XL is already unzipped
if os.path.exists(ptbxl_path) and not os.path.exists(ptbxl_extract_path):
    print("Unzipping PTB-XL dataset...")
    with zipfile.ZipFile(ptbxl_path, 'r') as zip_ref:
        zip_ref.extractall(ptbxl_extract_path)
    print("PTB-XL dataset unzipped.")
else:
    print("PTB-XL dataset already unzipped or ZIP file missing.")

# Check if SamiTrop is already unzipped
if os.path.exists(samitrop_path) and not os.path.exists(samitrop_extract_path):
    print("Unzipping SamiTrop dataset...")
    with zipfile.ZipFile(samitrop_path, 'r') as zip_ref:
        zip_ref.extractall(samitrop_extract_path)
    print("SamiTrop dataset unzipped.")
else:
    print("SamiTrop dataset already unzipped or ZIP file missing.")

# Update these paths to the extracted directories
ptbxl_records_dir = os.path.join(DATA_DIR, "ptbxl_output", "ptbxl_output")
samitrop_records_dir = os.path.join(DATA_DIR, "samitrop_output", "samitrop_output")


PTB-XL dataset already unzipped or ZIP file missing.
SamiTrop dataset already unzipped or ZIP file missing.


# Datasplitting

## Metadata Collection Helpers

In [9]:
def gather_file_metadata(subdir):
    hea_files = sorted([f for f in os.listdir(os.path.join(DATA_DIR, subdir)) if f.endswith(".hea")])
    data = []
    for hea_file in hea_files:
        base = hea_file.replace(".hea", "")
        dat_path = os.path.join(DATA_DIR, subdir, base + ".dat")
        hea_path = os.path.join(DATA_DIR, subdir, hea_file)

        if not os.path.exists(dat_path):
            continue

        header = load_header(hea_path)
        label = get_label(header)
        source = get_source(header)
        age = get_age(header)
        sex = get_sex(header)

        data.append({
            "record_id": base,
            "dat_path": dat_path,
            "hea_path": hea_path,
            "label": label,
            "source": source,
            "age": age,
            "sex": sex,
            "subdir": subdir
        })
    return pd.DataFrame(data)


## Directory Cleaners

In [10]:
def clean_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    else:
        for f in os.listdir(folder):
            if f.endswith(".dat") or f.endswith(".hea"):
                os.remove(os.path.join(folder, f))


## Combine + Split + Compute Class Weights

In [11]:
def combine_and_split_all():
    print("\n Removing files that was already there.")
    clean_dir(os.path.join(DATA_DIR, "train"))
    clean_dir(os.path.join(DATA_DIR, "val"))
    clean_dir(os.path.join(DATA_DIR, "test"))
    print("\n📆 Collecting all .hea/.dat files and labels")

    df_ptbxl = gather_file_metadata("ptbxl_output/ptbxl_output")
    df_samitrop = gather_file_metadata("samitrop_output/samitrop_output")

    df_all = pd.concat([df_ptbxl, df_samitrop], ignore_index=True)
    print(f"✅ Total records found: {len(df_all)}")
    print("Label distribution in full dataset:")
    print(df_all['label'].value_counts())

    df_trainval, df_test = train_test_split(df_all, test_size=0.15, stratify=df_all["label"], random_state=42)
    df_train, df_val = train_test_split(df_trainval, test_size=0.1765, stratify=df_trainval["label"], random_state=42)

    print("\n📊 Train:")
    print(df_train["label"].value_counts())
    print("📊 Val:")
    print(df_val["label"].value_counts())
    print("📊 Test:")
    print(df_test["label"].value_counts())

    y_train = df_train["label"].values
    class_weights = compute_class_weight(class_weight="balanced", classes=np.array([0, 1]), y=y_train)
    weight_dict = {0: class_weights[0], 1: class_weights[1]}

    print("\n✅ Class weights:")
    print(weight_dict)

    return df_train, df_val, df_test, weight_dict


## Save Records to Split Folders

In [12]:
def save_records(df, out_dir):
    clean_dir(out_dir)
    for _, row in df.iterrows():
        base = row["record_id"]
        subdir = row["subdir"]
        for ext in [".hea", ".dat"]:
            src = os.path.join(DATA_DIR, subdir, base + ext)
            dst = os.path.join(out_dir, base + ext)
            if os.path.exists(src):
                shutil.copy(src, dst)


## Run Full Split + Save Folders

In [13]:
df_train, df_val, df_test, weight_dict = combine_and_split_all()

save_records(df_train, os.path.join(DATA_DIR, "train"))
save_records(df_val, os.path.join(DATA_DIR, "val"))
save_records(df_test, os.path.join(DATA_DIR, "test"))





 Removing files that was already there.

📆 Collecting all .hea/.dat files and labels
✅ Total records found: 1659
Label distribution in full dataset:
label
1    1631
0      28
Name: count, dtype: int64

📊 Train:
label
1    1141
0      20
Name: count, dtype: int64
📊 Val:
label
1    245
0      4
Name: count, dtype: int64
📊 Test:
label
1    245
0      4
Name: count, dtype: int64

✅ Class weights:
{0: np.float64(29.025), 1: np.float64(0.5087642418930762)}


## Create Balanced Sampler & Train Loader

In [21]:
from torch.utils.data import WeightedRandomSampler

# --- Custom create_balanced_sampler ---
def create_balanced_sampler(dataset):
    labels = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        labels.append(int(label.item()))
    labels = np.array(labels)

    label_counts = np.bincount(labels)
    class_weights = 1.0 / label_counts
    sample_weights = class_weights[labels]
    return WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# --- Usage ---
train_dataset = ECGFromCSV('/content/drive/MyDrive/chagas_datasets/train', augment=True)
sampler = create_balanced_sampler(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler)


In [22]:
from collections import Counter
samples = [df_train['label'].iloc[i] for i in list(sampler)]
print(Counter(samples))

Counter({np.int64(1): 595, np.int64(0): 566})


# Train, run, and evaluate model

In [23]:
!python3 -c "import torch; print(torch.cuda.is_available())"


False


## Train

In [24]:
!python train_model.py -d /content/drive/MyDrive/chagas_datasets/train -m model -v

💻 Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100% 97.8M/97.8M [00:00<00:00, 112MB/s]
Training audio_classifier...
audio_classifier - Epoch 1, Loss: 0.2553
Traceback (most recent call last):
  File "/content/drive/MyDrive/chagas-challenge/train_model.py", line 32, in <module>
    run(get_parser().parse_args(sys.argv[1:]))
  File "/content/drive/MyDrive/chagas-challenge/train_model.py", line 29, in run
    train_model(args.data_folder, args.model_folder, args.verbose) ### Teams: Implement this function!!!
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/chagas-challenge/team_code.py", line 139, in train_model
    out = model(x, x1) if model_name == 'resnet' else model(x)
                                                      ^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_ca

## Run

In [ ]:
!python run_model.py -d /content/drive/MyDrive/chagas_datasets/val -m model -o /content/drive/MyDrive/chagas_datasets/holdout_outputs -v

## Test

In [ ]:
!python evaluate_model.py -d /content/drive/MyDrive/chagas_datasets/test -o /content/drive/MyDrive/chagas_datasets/holdout_outputs -s scores.csv